1.準備

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install tensorflow tensorflow-gpu  matplotlib
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import display
import pandas as pd

import librosa
import librosa.display

import tensorflow as tf 
import tensorflow_io as tfio

In [ ]:
# pyplotのデフォルト値を設定
plt.rcParams.update({
  'font.size' : 10
  ,'font.family' : 'Meiryo' if os.name == 'nt' else ''  # Colabでは日本語フォントがインストールされてないので注意
  ,'figure.figsize' : [10.0, 5.0]
  ,'figure.dpi' : 300
  ,'savefig.dpi' : 300
  ,'figure.titlesize' : 'large'
  ,'legend.fontsize' : 'small'
  ,'axes.labelsize' : 'medium'
  ,'xtick.labelsize' : 'small'
  ,'ytick.labelsize' : 'small'
  })
# ndarrayの表示設定
np.set_printoptions(threshold=0)  # 可能ならndarrayを省略して表示
np.set_printoptions(edgeitems=1)  # 省略時に１つの要素だけ表示

2.短時間フーリエ変換

In [ ]:
# STFT (s: signal(1D-array), Lf: length of frame(window), noverlap: number of overlap)
def STFT(s, Lf, noverlap=None):
    if noverlap==None:
        noverlap = Lf//2
    l = s.shape[0]
    win = np.hanning(Lf)
    Mf = Lf//2 + 1
    Nf = int(np.ceil((l-noverlap)/(Lf-noverlap)))-1
    S = np.empty([Mf, Nf], dtype=np.complex128)
    for n in tqdm(range(Nf)):
        S[:,n] = np.fft.rfft(s[(Lf-noverlap)*n:(Lf-noverlap)*n+Lf] * win, n=Lf, axis=0)
    return S

In [ ]:
import soundfile as sf

# データの読み込み

person = 6

SS = [[] for i in range(person)] 

for i in range(person+1):
  # p=i+6
  if i < 10:
    SS[i-1] = os.path.join('/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/', 's0' + str(i))
  else:
    SS[i-1] = os.path.join('/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/', 's' + str(i))

# for i in range(person):
#   print(SS[i])

print(SS[0])
print(SS[5])


/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/s01
/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/s06


In [ ]:
kankyou = 6
yy = [[] for i in range(person * kankyou)] 
j = 0

print(yy)
for n in range(person):

  for i in os.listdir(SS[n]):
    file_name = SS[n] + '/' + i

    if j == 0:
      y, sr0 = sf.read(file_name)
      print('サンプリング周波数 = ',sr0)
      sr = sr0

    else:
      y, sr = sf.read(file_name)
    
    yy[j] = y
    print(i)

    if sr != sr0:
      print('サンプリング周波数の異なるファイルがあります')
      print(sr)

    j += 1


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
サンプリング周波数 =  16000
s01_2_footstep_audio.wav
s01_1_footstep_audio.wav
s01_4_footstep_audio.wav
s01_3_footstep_audio.wav
s01_1_footstep_audio_HPSS.wav
s01_2_footstep_audio_HPSS.wav
s01_3_footstep_audio_HPSS.wav


LibsndfileError: ignored

In [ ]:
spec = [[] for i in range(person * kankyou)]
for j in range(person * kankyou):
  spec[j] = STFT(yy[j], 1024)

# spec = STFT(y, 1024)
# spec_db = librosa.amplitude_to_db(np.abs(spec))
# librosa.display.specshow(spec_db, x_axis='time', y_axis="log")


In [ ]:
# display(IPython.display.Audio(y, rate=sr))

3.調波打楽器音分離

In [ ]:
# 源信号の短時間フーリエ変換の絶対値abs(stft)を入力し、
# 調波音、打楽器音を分離するためのウィーナーフィルターH_mask, P_maskを計算する。

# H = [[] for i in range(person * kankyou)]
P = [[] for i in range(person * kankyou)]

for i in range(person * kankyou):
  H_mask, P_mask = librosa.decompose.hpss(abs(spec[i]), kernel_size=(51,51), mask=True)
  # H[i] = H_mask * spec[i]
  P[i] = P_mask * spec[i]



In [ ]:
from scipy.signal import istft

yp_inv = [[] for i in range(person * kankyou)]

for i in range(person * kankyou):
  yp_inv[i] = istft(P[i], sr)
  yp_inv[i] = yp_inv[i][1]
  yp_inv[i] = yp_inv[i] / max(yp_inv[i]) * max(yy[i])

  if max(yp_inv[i]) > max(yy[i]):
    print('error')

In [ ]:
import soundfile
j = 0

for i in range(person * kankyou):
  # p=i+6
  j = int(i / kankyou) #+6
  print(j)
  k = i % kankyou

  if j < 9:
    soundfile.write(file=os.path.join('/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/s0'+str(j+1)+'/s0'+str(j+1)+'_'+str(k+1)+'_footstep_audio_HPSS.wav'), data=yp_inv[i], samplerate=sr)

  else:
     soundfile.write(file=os.path.join('/content/drive/MyDrive/audio_processing/intern/AFPID-II/AFPID-II_P2-Raw/s'+str(j+1)+'/s'+str(j+1)+'_'+str(k+1)+'_footstep_audio_HPSS.wav'), data=yp_inv[i], samplerate=sr)
